In [10]:
import numpy as np
from keras.layers import Input, Conv2D, Flatten, Dense, LeakyReLU, Conv2DTranspose, Reshape, Activation
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam
from keras.datasets import mnist

In [11]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.astype('float32') / 255.
x_test = x_test.reshape(x_test.shape + (1,))

Create encoder model

In [12]:
encoder_input = Input((28, 28, 1))

e = Conv2D(
    filters=32,
    kernel_size=3,
    strides=1,
    padding='same'
    )(encoder_input)
e = LeakyReLU()(e)

e = Conv2D(
    filters=64,
    kernel_size=3,
    strides=2,
    padding='same'
    )(e)
e = LeakyReLU()(e)

e = Conv2D(
    filters=64,
    kernel_size=3,
    strides=2,
    padding='same'
    )(e)
e = LeakyReLU()(e)

e = Conv2D(
    filters=64,
    kernel_size=3,
    strides=1,
    padding='same'
    )(e)
e = LeakyReLU()(e)

shape_before_flattening = K.int_shape(e)[1:]

e = Flatten()(e)
encoder_output = Dense(2)(e)

encoder_model = Model(encoder_input, encoder_output)

encoder_model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 7, 7, 64)          0   

Create decoder model

In [13]:
decoder_input = Input((2,))

d = Dense(np.prod(shape_before_flattening))(decoder_input)
d = Reshape(shape_before_flattening)(d)

d = Conv2DTranspose(
    filters=64,
    kernel_size=3,
    strides=1,
    padding='same'
    )(d)
d = LeakyReLU()(d)

d = Conv2DTranspose(
    filters=64,
    kernel_size=3,
    strides=2,
    padding='same'
    )(d)
d = LeakyReLU()(d)

d = Conv2DTranspose(
    filters=32,
    kernel_size=3,
    strides=2,
    padding='same'
    )(d)
d = LeakyReLU()(d)

d = Conv2DTranspose(
    filters=1,
    kernel_size=3,
    strides=1,
    padding='same'
    )(d)
d = Activation('sigmoid')(d)

decoder_output = d

decoder_model = Model(decoder_input, decoder_output)

decoder_model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 3136)              9408      
_________________________________________________________________
reshape_2 (Reshape)          (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose_5 (Conv2DTr (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 14, 14, 64)        0   

Full autoencoder

In [14]:
ae_input = encoder_input
ae_output = decoder_model(encoder_output)

ae_model = Model(ae_input, ae_output)

Training

In [15]:
LEARNING_RATE = 0.0005
BATCH_SIZE = 32
EPOCHS = 2
INITIAL_EPOCH = 0

In [16]:
def r_loss(y_true, y_pred):
    return K.mean(K.square(y_true - y_pred), axis = [1,2,3])

In [17]:
optimizer = Adam(lr=LEARNING_RATE)
ae_model.compile(optimizer=optimizer, loss=r_loss)

In [18]:
ae_model.fit(     
    x_train,
    x_train,
    batch_size=BATCH_SIZE,
    shuffle=True,
    epochs=EPOCHS,
    initial_epoch=INITIAL_EPOCH,
    #callbacks=callbacks_list
    )

Epoch 1/2
60000/60000 [==============================] - 200s 3ms/step - loss: 0.0550
Epoch 2/2
42144/60000 [====================>.........] - ETA: 1:00 - loss: 0.0463

KeyboardInterrupt: ignored

In [29]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [35]:
ae_model.save("/content/gdrive/My Drive/data/models/ae_test_save.model")

In [36]:

import os
print( os.getcwd() )
#print( os.listdir('data') )
os.listdir('/content/gdrive/My Drive/data/models')

/content


['ae_test_save.model']